In [1]:
import bilby
import gwpopulation as gwpop
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import corner

from astropy import cosmology, units
from scipy.interpolate import interp1d

from bilby.core.prior import Uniform, LogUniform, PriorDict, Constraint
from gwpopulation.models import mass, spin

xp = gwpop.cupy_utils.xp

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
parameter_translator = dict(
    mass_1_det = 'm1_detector_frame_Msun',
    mass_2_det = 'm2_detector_frame_Msun',
    luminosity_distance = 'luminosity_distance_Mpc',
    a_1 = 'spin1',
    a_2 = 'spin2',
    cos_tilt_1 = 'costilt1',
    cos_tilt_2 = 'costilt2')

In [3]:
posteriors = list()

file = 'GWTC-1_sample_release/GW{}_GWTC-1.hdf5'

events = ['150914', '151012', '151226', '170104', '170608', '170729', '170809', '170814', 
          '170818', '170823']

for event in events:
    _posterior = pd.DataFrame()
    with h5py.File(file.format(event), 'r') as ff:
        for my_key, gwtc_key in parameter_translator.items():
            _posterior[my_key] = ff['IMRPhenomPv2_posterior'][gwtc_key]
    posteriors.append(_posterior)

In [4]:
luminosity_distances = np.linspace(1, 10000, 1000)
redshifts = np.array([cosmology.z_at_value(cosmology.Planck15.luminosity_distance, 
                                                   dl * units.Mpc) for dl in luminosity_distances])
dl_to_z = interp1d(luminosity_distances, redshifts)

In [5]:
for posterior in posteriors:
    posterior['redshift'] = dl_to_z(posterior['luminosity_distance'])
    posterior['mass_1'] = posterior['mass_1_det'] / (1 + posterior['redshift'])
    posterior['mass_2'] = posterior['mass_2_det'] / (1 + posterior['redshift'])
    posterior['mass_ratio'] = posterior['mass_2'] / posterior['mass_1']

In [7]:
modelA = bilby.hyper.model.Model([mass.SinglePeakSmoothedMassDistribution(),
                                  spin.iid_spin])           #independent_spin_magnitude_beta])

### Default Model Parameters 

| $\alpha$ | $m_{max}$ | $m_{min}$ | $\beta_q$ | $\lambda_m$ | $\mu_m$ | $\sigma_m$ | $\delta_m$ | E[a] | Var[a] |   $\chi$ | $\sigma_a$ | 
|----------|-----------|-----------|-----------|-------------|---------|------------|------------|------|-------|----------|------------|
| [-4, 12] | [30, 100] | [5, 10] | [-4, 12] | [0, 1] | [20, 50] | (0, 10] | [0, 10] | [0, 1] | [0, 0.25] | [0, 1] | [0, 4] |

In [16]:
def convert_to_beta_parameters(parameters, remove=True):
    """
    Convert to parameters for standard beta distribution
    """
    added_keys = list()
    converted = parameters.copy()

    def _convert(suffix):
        alpha = f"alpha_chi{suffix}"
        beta = f"beta_chi{suffix}"
        mu = f"mu_chi{suffix}"
        sigma = f"sigma_chi{suffix}"
        amax = f"amax{suffix}"

        if alpha not in parameters.keys() or beta not in parameters.keys():
            needed = True
        elif converted[alpha] is None or converted[beta] is None:
            needed = True
        else:
            needed = False
            done = True

        if needed:
            if mu in converted.keys() and sigma in converted.keys():
                done = True
                (
                    converted[alpha],
                    converted[beta],
                    _,
                ) = mu_chi_var_chi_max_to_alpha_beta_max(
                    parameters[mu], parameters[sigma], parameters[amax]
                )
                if remove:
                    added_keys.append(alpha)
                    added_keys.append(beta)
            else:
                done = False
        return done

    done = False

    for suffix in ["_1", "_2"]:
        _done = _convert(suffix)
        done = done or _done
    if not done:
        _ = _convert("")
        
#     print("hi")

    return converted, added_keys


def alpha_beta_max_to_mu_chi_var_chi_max(alpha, beta, amax):
    """
    Convert between parameters for beta distribution
    """
    mu_chi = alpha / (alpha + beta) * amax
    var_chi = alpha * beta / ((alpha + beta) ** 2 * (alpha + beta + 1)) * amax ** 2
    return mu_chi, var_chi, amax


def mu_chi_var_chi_max_to_alpha_beta_max(mu_chi, var_chi, amax):
    """
    Convert between parameters for beta distribution
    """
#     print("hello")
    mu_chi /= amax
    var_chi /= amax ** 2
    alpha = (mu_chi ** 2 * (1 - mu_chi) - mu_chi * var_chi) / var_chi
    beta = (mu_chi * (1 - mu_chi) ** 2 - (1 - mu_chi) * var_chi) / var_chi
    return alpha, beta, amax

In [17]:
# from gwpopulation.conversions import convert_to_beta_parameters

def prior_conversion(parameters):
    """Wrapper around conversion for prior constraints"""
    for key in ["amax_1", "amax_2"]:
#         print("hello")
        if key not in parameters:
            parameters[key] = 1
    parameters, _ = convert_to_beta_parameters(parameters)
    return parameters

In [18]:
priorsA = bilby.gw.prior.BBHPriorDict(conversion_function = prior_conversion)
          
# mass
priorsA["alpha"] = Uniform(minimum = -4, maximum = 12, latex_label = "$\\alpha$")
priorsA["beta"] = Uniform(minimum = -4, maximum = 12, latex_label = "$\\beta$")
priorsA["mmin"] = Uniform(minimum = 5, maximum = 10, latex_label = "$M_{min}$")
priorsA["mmax"] = Uniform(minimum = 30, maximum = 100, latex_label = "$M_{max}$")
priorsA["lam"] = Uniform(minimum = 0, maximum = 1, latex_label = "$\\lambda$")
priorsA["mpp"] = Uniform(minimum=20, maximum=50, latex_label='$\\mu_{m}$')
priorsA["sigpp"] = Uniform(minimum=0, maximum=10, latex_label='$\\sigma_{m}$') # sigpp != 0
priorsA["delta_m"] = Uniform(minimum=0, maximum=10, latex_label='$\\delta_{m}$') 

# spin magnitude 
priorsA["mu_chi_1"] = Uniform(minimum=0, maximum=1, latex_label = '$\\mu_{\\chi_1}$')
priorsA["mu_chi_2"] = Uniform(minimum=0, maximum=1, latex_label = '$\\mu_{\\chi_2}$')
priorsA["sigma_chi_1"] = Uniform(minimum=0, maximum=0.25, latex_label = '$\\sigma_{\\chi_1}$')
priorsA["sigma_chi_2"] = Uniform(minimum=0, maximum=0.25, latex_label = '$\\sigma_{\\chi_2}$')

# spin orientation
priorsA["xi_spin"] = Uniform(minimum=0, maximum=1, latex_label='$\\xi$')
priorsA["sigma_1"] = Uniform(minimum=0, maximum=4, latex_label='$\\sigma{1}$')
priorsA["sigma_2"] = Uniform(minimum=0, maximum=4, latex_label='$\\sigma{2}$')

11:39 bilby INFO    : No prior given, using default BBH priors in /home/deepnika.jain/anaconda3/envs/gwspin/lib/python3.8/site-packages/bilby/gw/prior_files/precessing_spins_bbh.prior.


In [19]:
prior_samples = priorsA.sample(100)

KeyError: 'amax_1'

In [ ]:
def chieff(a1, a2, q, cost1, cost2):
    chi_eff = (a1*cost1 + a2*q*cost2)/(1+q)
    return chi_eff

def chiP(a1, a2, q, cost1, cost2):
    q1 = a1 * sin(arccos(cost1))
    q2 = ((4*q + 3)/(4 + 3*q)) * q * a2 * sin(arccos(cost2))
    return np.max([q1, q2])